In [1]:
from ML_utils import ML_utils
import numpy as np

In [2]:
dir_path = r'D:\Backup\ml_data\GitHub\ML_Sentiment_analysis\Dataset'
x = ML_utils.CallF_DirFile(dir_path,ML_utils.WordToList_file,max_word_num=60,padding=0,file_filter_='sentiment')[0]
y = np.concatenate( (np.full(1000,0),np.full(1000,1)) )

In [ ]:

class Dataset(data.Dataset):
    def __init__(self,x,y,model_path,transform=None,target_transform=None,seq_len=30):
        """
        seq_len: Trim length to seq_len
        """
        self.transform = transform
        self.target_transform = target_transform
        file_path = file_path
        self.chat_data = self.trim(file_path,max_word_num=seq_len)
        self.word_vec_path = model_path
        self.word_vec_model = word2vec.Word2Vec.load(self.word_vec_path)
        #padding(Null / other)
        self.padding = np.zeros((1,300),dtype=np.float32)
#         self.padding = np.array(self.word_vec_model.wv.get_vector('。')).reshape(1,300)
        
    def __getitem__(self, index):
        X,Y = self.word_to_vec(index)
        return X, Y
                
    def __len__(self):
        return len(self.chat_data)
    
    def trim(self,file_path,max_word_num):
        with open(file_path,'r',encoding='utf-8') as f:
            new_list = []
            for chat in f:
                #依行讀取
                save_chat = chat.strip('\n').split('\t')
                X = save_chat[0].split(' ')
                Y = save_chat[1].split(' ')
                #如果詞數大於max_word_num
                if len(X) < max_word_num and len(Y) < max_word_num:
                    for num in range(max_word_num-len(X)):
                        X.append(0)
                    for num in range(max_word_num-len(Y)):
                        Y.append(0)
                    new_list.append([X,Y])
                if len(save_chat) < 2:
                    print(save_chat)
                    break
            num = len(new_list)//100
            new_list = new_list[:num*100]
        return new_list
    
    def word_to_vec(self,index):
        chat = self.chat_data[index]
        #X
        vec_array = np.zeros((1,300),dtype=np.float32)
        for word in chat[0]:
            if word == 0:
                vec = self.padding
                vec_array = np.concatenate((vec_array,vec),axis=0)
            else:
                try:
                    vec = np.array(self.word_vec_model.wv.get_vector(word),np.float32).reshape(1,300)
                    vec_array = np.concatenate((vec_array,vec),axis=0)
                except KeyError:
                    vec = self.padding
                    vec_array = np.concatenate((vec_array,vec),axis=0)
        X = vec_array[1:]
        #Y
        vec_array = np.zeros((1,300),dtype=np.float32)
        for word in chat[1]:
            if word == 0:
                vec = self.padding
                vec_array = np.concatenate((vec_array,vec),axis=0)
            else:
                try:
                    vec = np.array(self.word_vec_model.wv.get_vector(word),np.float32).reshape(1,300)
                    vec_array = np.concatenate((vec_array,vec),axis=0)
                except KeyError:
                    vec = self.padding
                    vec_array = np.concatenate((vec_array,vec),axis=0)
        Y = vec_array[1:]
        return X,Y

In [ ]:
seq_len = 60